AWS

In [1]:
import boto3
from PIL import Image
import os
import pandas as pd
import io
import numpy as np
import re
import time

In [2]:
DIR_PATH = "/users/Shared/New"

In [3]:
client=boto3.client('rekognition')

In [4]:
def detect_text_aws(img, client):
    img.save("new.png")
    with io.open("./new.png", 'rb') as image_file: # use file.filename
        response = client.detect_text(Image={'Bytes': image_file.read()})

    return response

In [5]:
def get_heroes_stats(player_name, img, response, all_heroes_df):
    all_words_in_image = pd.DataFrame([(x['DetectedText'], x['Geometry']['Polygon'][0]['X'], x['Geometry']['Polygon'][0]['Y'], i) for i,x in enumerate(response['TextDetections']) if x['Type'] == 'WORD'], columns=['Word', 'x', 'y', 'PosNum'])

    result_df = all_heroes_df.merge(all_words_in_image, how = 'left', on = 'Word')
    result_df = result_df[result_df['PosNum'].notna()]
    result_df['PosDiff'] = result_df['PosNum'] - result_df['WordNum']
    result_df = result_df.groupby(['Hero', 'PosDiff']).agg({'WordCount': 'max', 'Word': 'count', 'x':'min', 'y': 'min'})
    boxes = result_df[result_df['WordCount'] == result_df['Word']].reset_index()[['Hero', 'x', 'y']].values.tolist()

    return [
        (player_name,)
        +(box[0],)
        +([s['DetectedText'] for s in response['TextDetections'] if s['Type'] == 'WORD' and np.abs(s['Geometry']['Polygon'][0]['X'] - box[1] - 0.0036100447177886963  / 0.74) < 0.01 and np.abs(s['Geometry']['Polygon'][0]['Y'] - box[2] + 0.30078125 / 0.8) < 0.01][0:1],)
        +([s['DetectedText'] for s in response['TextDetections'] if s['Type'] == 'WORD' and np.abs(s['Geometry']['Polygon'][0]['X'] - box[1] - 0.041064225137233734  / 0.74) < 0.015 and np.abs(s['Geometry']['Polygon'][0]['Y'] - box[2] + 0.1259765625 / 0.8) < 0.015][0:1],)
        + get_stars(img, box[1], box[2])
        for box in boxes
    ]

In [6]:
def get_star(s):
    if s[0] < 100:
        return 0
    elif s[2] < s[1]:
        return 1
    else:
        return 2

def get_stars(img, x, y):
    q = 0.01495 / 0.74
    stars_list = [get_star(img.getpixel((img.width * (x + 0.001 / 0.74 + i * q), img.height * (y - 0.078 / 0.8)))) for i in range(7)]
    return sum([1 for x in stars_list if x == 2]), sum([1 for x in stars_list if x >= 1])

Get hero power

In [7]:
bad_list = []
responses = {}

In [8]:
all_heroes_list = pd.read_csv("all_heroes.csv", header=None)[0].values.tolist()
all_heroes_df = pd.DataFrame(
    [item for sublist in [[(x, y, i, len(x.split())) for i, y in enumerate(x.split())] for x in all_heroes_list] for
     item in sublist], columns=['Hero', 'Word', 'WordNum', 'WordCount'])

In [13]:
heroes_list = []
player_name = ""
for (dirpath, dirnames, filenames) in os.walk(DIR_PATH):
    filenames.sort()
    for file in filenames:
        if file != ".DS_Store":
            ##################################
            ##Change the criterion for distinguishing player screen from main screens there
            ##################################
            if os.path.getsize(dirpath + "/" + file) <= 4.5 * 1024 * 1024:
                img = Image.open(dirpath+'/'+file)
                response1 = detect_text_aws(img.crop((500, 0, 1800, 180)), client)
                player_name = response1["TextDetections"][0]["DetectedText"]
                print('switching player name to ' + player_name)
            else:
                img = Image.open(dirpath+'/'+file)
                img = img.crop((img.width * 0.09, img.height * 0.2, img.width * 0.83, img.height))
                img.save("new.png")
                with io.open("./new.png", 'rb') as image_file: # use file.filename
                    response = client.detect_text(Image={'Bytes': image_file.read()})
                responses[file] = response
                #response = responses[file]
                heroes_list_new = get_heroes_stats(player_name, img, response, all_heroes_df)
                heroes_list += heroes_list_new
                if len(heroes_list_new) != 10:
                    bad_list.append((file, heroes_list_new))
                    print("Not all heroes from " + file + " parsed. Only " + str(len(heroes_list_new)) +"/10 heroes parsed. You may find AWS recognition response for that file in responses dict with filename as the key")

switching player name to Moore#12TO


In [14]:
df_heroes = pd.DataFrame(heroes_list, columns = ['Player', 'Hero', 'Level', 'Eq', 'AS', 'Stars'])
df_heroes['Level'] = df_heroes['Level'].astype(str).str.extract('([0-9]+)')
df_heroes['Eq'] = df_heroes['Eq'].astype(str).str.replace('([^0-9]+)', '', regex = True)
df_heroes.to_excel("player_hero.xlsx")

Raid results

In [ ]:
p = re.compile('(.+#.+)|([0-9,]+)')
player_results = []
for (dirpath, dirnames, filenames) in os.walk(DIR_PATH):
    filenames.sort()
    for file in filenames:
        if file != ".DS_Store":
            img = Image.open(DIR_PATH + '/' + file)
            img = img.crop((img.width * 0.315, img.height * 0.42, img.width * 0.83 , img.height))
            img.save("new.png")
            with io.open("./new.png", 'rb') as image_file: # use file.filename
                response = client.detect_text(Image={'Bytes': image_file.read()})
            s = [x for x in [(x['DetectedText']) for x in response['TextDetections'] if x['Type'] == 'WORD'] if p.match(x)]
            player_results += [(time.ctime(os.path.getmtime(DIR_PATH + '/' + file)), s[2*i], s[i*2+1]) for i in range(int(len(s)/2))]

In [ ]:
pd.DataFrame(player_results).to_excel("ER_results2.xlsx")

Checks (used only for debugging)

In [ ]:
img = Image.open(DIR_PATH + '/' + "IMG_0085.PNG")
img = img.crop((img.width * 0.315, img.height * 0.42, img.width * 0.83 , img.height))
img.save("new.png")

In [ ]:
s = [(x['DetectedText']) for x in response['TextDetections'] if x['Type'] == 'WORD']

In [ ]:
p = re.compile('(.+#.+)|([0-9,]+)')

In [ ]:
s = [x for x in [(x['DetectedText']) for x in response['TextDetections'] if x['Type'] == 'WORD'] if p.match(x)]

In [ ]:
s

In [ ]:
[(s[2*i], s[i*2+1]) for i in range(int(len(s)/2))]

In [ ]:
player_results

In [ ]:
img = Image.open(DIR_PATH + '/' + "IMG_0085.PNG")
img = img.crop((img.width * 0.09, img.height * 0.2, img.width * 0.83 , img.height))
img.save("new.png")

In [ ]:
img

In [ ]:
with io.open("./new.png", 'rb') as image_file: # use file.filename
    response = client.detect_text(Image={'Bytes': image_file.read()})

In [ ]:
response = responses['IMG_0013.PNG']

In [ ]:
get_heroes_stats('Victor', img, response, all_heroes_df)

In [ ]:
img

In [ ]:
all_heroes_list = pd.read_csv("all_heroes.csv", header=None)[0].values.tolist()

In [ ]:
all_heroes_df = pd.DataFrame([item for sublist in [[(x, y, i, len(x.split())) for i, y in enumerate(x.split())] for x in all_heroes_list] for item in sublist], columns=['Hero', 'Word', 'WordNum', 'WordCount'])

In [ ]:
all_words_in_image = pd.DataFrame([(x['DetectedText'], x['Geometry']['Polygon'][0]['X'], x['Geometry']['Polygon'][0]['Y'], i) for i,x in enumerate(response['TextDetections']) if x['Type'] == 'WORD'], columns=['Word', 'x', 'y', 'PosNum'])

In [ ]:
all_words_in_image

In [ ]:
result_df = all_heroes_df.merge(all_words_in_image, how = 'left', on = 'Word')
result_df = result_df[result_df['PosNum'].notna()]
result_df['PosDiff'] = result_df['PosNum'] - result_df['WordNum']
result_df = result_df.groupby(['Hero', 'PosDiff']).agg({'WordCount': 'max', 'Word': 'count', 'x':'min', 'y': 'min'})
result_df = result_df[result_df['WordCount'] == result_df['Word']].reset_index()[['Hero', 'x', 'y']].values.tolist()

In [ ]:
[x for x in result_df]

In [ ]:
img.crop()

In [ ]:
boxes = [(s['DetectedText'], s['Geometry']['Polygon'][0]['X'], s['Geometry']['Polygon'][0]['Y']) for s in response['TextDetections'] if s['DetectedText'] in all_heroes_list and s['Type'] == 'LINE']
boxes

In [ ]:
[
    (box[0],)
    +([s['DetectedText'] for s in response['TextDetections'] if s['Type'] == 'WORD' and np.abs(s['Geometry']['Polygon'][0]['X'] - box[1] - 0.0036100447177886963  / 0.7) < 0.01 and np.abs(s['Geometry']['Polygon'][0]['Y'] - box[2] + 0.30078125 / 0.8) < 0.01][0:1],)
    +([s['DetectedText'] for s in response['TextDetections'] if s['Type'] == 'WORD' and np.abs(s['Geometry']['Polygon'][0]['X'] - box[1] - 0.041064225137233734  / 0.7) < 0.015 and np.abs(s['Geometry']['Polygon'][0]['Y'] - box[2] + 0.1259765625 / 0.8) < 0.015][0:1],)
    + get_stars(img, box[1], box[2])
    for box in boxes
]

In [ ]:
img.crop((img.width * (boxes[1][1] - 0.01), img.height * (boxes[1][2] - 0.1), img.width * (boxes[1][1] + 0.1), img.height * (boxes[1][2] - 0.06)))

In [ ]:
x = boxes[0][1]
y = boxes[0][2]

[img.getpixel((img.width * (x + 0.001 + i * q), img.height * (y - 0.078))) for i in range(7)]

In [ ]:
[s['DetectedText'] for s in response['TextDetections']]